In [ ]:
#Unsupervised Sentiment Analysis - Lexicon based models
1. TextBlob based Sentiment Analysis
2. SentiWordNet Lexicon
3. AFINN based
4. Vader based

In [1]:
!pip install textblob
!pip install textsearch
!pip install contractions 
!pip install afinn
import nltk 
nltk.download('all')

     |████████████████████████████████| 287 kB 5.2 MB/s 
     |████████████████████████████████| 106 kB 61.3 MB/s 
     |████████████████████████████████| 52 kB 1.1 MB/s 
  Created wheel for afinn: filename=afinn-0.1-py3-none-any.whl size=53447 sha256=d13d3ff67d2c10d66f11813b662d3805dc149333e4160c925478d29be64e6435
  Stored in directory: /root/.cache/pip/wheels/9d/16/3a/9f0953027434eab5dadf3f33ab3298fa95afa8292fcf7aba75
Successfully built afinn
[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /

True

In [7]:
import pandas as pd 
import numpy as np 
import nltk 
import textblob
import contractions
from sklearn.metrics import confusion_matrix, classification_report 
from sklearn.metrics import accuracy_score

In [ ]:
data = pd.read_csv("movie_reviews.csv.bz2")

In [4]:
data.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
data.shape

(50000, 2)

In [6]:
reviews = np.array(data['review'])
sentiments = np.array(data['sentiment'])

In [15]:
from bs4 import BeautifulSoup 
import tqdm
import re
import unicodedata

def strip_html_tags(text):
  soup = BeautifulSoup(text,'html.parser')
  stripped_text = soup.get_text()
  stripped_text = re.sub(r'[\r|\n|\r\n]+','\n',stripped_text)
  return stripped_text

In [ ]:
strip_html_tags(data['review'][1])

In [18]:
stop_words = nltk.corpus.stopwords.words('english')

In [28]:
def remove_accented_chars(text):
  text = unicodedata.normalize('NFKD',text).encode('ascii','ignore').decode('utf-8','ignore')
  return text

In [62]:
import string
def pre_process_corpus(doc):
  norm_corpus = []
  for text in tqdm.tqdm(doc):
    text = strip_html_tags(text)
    text = remove_accented_chars(text)
    text = re.sub(r'[^a-zA-Z0-9\s]',' ',text,re.I)
    text = text.translate(str.maketrans('','',string.punctuation))
    text = text.lower()
    text = text.strip()
    text = contractions.fix(text)
    tokens = nltk.word_tokenize(text)
    filtered_words = [token for token in tokens if token not in stop_words]
    text = " ".join(filtered_words)
    text = re.sub(' +',' ',text)
    norm_corpus.append(text)
  return norm_corpus

In [63]:
pre_process_corpus([test_reviews[1500],''])

100%|██████████| 2/2 [00:00<00:00, 431.56it/s]


['sure comment contains actual spoiler playing safe read seen movie adore movie everyone work point spent large part working life cinema without actor people sung heroes movie gaffers pullers onair directors lighters writers costume people etc etc whole thing told point view least great extent actors nuts selfabsorbed point absurdity movie spoofs well worked actors recognize movie reallife drama possible spoiler alert one great scene two leads actors discussing discuss something personal something entirely outofscript another actor start making lines rehearsing critiquing performancesince movie appeared 91 become fashionable especially tv hardly anyone done well',
 '']

In [64]:
%%time

normalized_reviews = pre_process_corpus(reviews)

100%|██████████| 50000/50000 [01:30<00:00, 554.37it/s]

CPU times: user 1min 29s, sys: 641 ms, total: 1min 29s
Wall time: 1min 30s


In [68]:
#Textblob
test_reviews = reviews[35000:]
test_sentiments = sentiments[35000:]
sample_review_ids = [1500,2500,13010]

In [40]:
#Without Pre-processing
for review, sentiment in zip(test_reviews[sample_review_ids],test_sentiments[sample_review_ids]):
  print('REVIEW:',review)
  print('Actual Sentiment:',sentiment)
  print("Predicted Sentiment:", textblob.TextBlob(review).sentiment.polarity)
  print('-'*100)

REVIEW: I'm not sure that this comment contains an actual spoiler, but I'm playing it safe, so don't read this if you haven't seen the movie.<br /><br />I adore this movie, and so does everyone I work with -- and that is the point. I spent a large part of my working life in cinema, without being an actor. Such people are the _sung_ heroes of this movie: the gaffers, the pullers, the on-air directors, the lighters and writers, the costume people etc etc, and the whole thing is told from their point of view, at least to a great extent. Most actors are nuts and self-absorbed to the point of absurdity, which is what this movie spoofs so well, but you have to have worked with actors to recognize that this movie is real-life drama! Possible spoiler alert: in one great scene, the two leads, both actors, are _discussing_ how to _discuss_ something personal, something entirely 'out-of-script', with another actor, and they start making up lines, rehearsing them, and critiquing each other's perfo

In [43]:
#Textblob
test_reviews = np.array(normalized_reviews[35000:])
test_sentiments = sentiments[35000:]
sample_review_ids = [1500,2500,13010]

In [44]:
#Without Pre-processing
for review, sentiment in zip(test_reviews[sample_review_ids],test_sentiments[sample_review_ids]):
  print('REVIEW:',review)
  print('Actual Sentiment:',sentiment)
  print("Predicted Sentiment:", textblob.TextBlob(review).sentiment.polarity)
  print('-'*100)

REVIEW: sure comment contains actual spoiler playing safe , read seen movie.i adore movie , everyone work -- point . spent large part working life cinema , without actor . people _sung_ heroes movie : gaffers , pullers , on-air directors , lighters writers , costume people etc etc , whole thing told point view , least great extent . actors nuts self-absorbed point absurdity , movie spoofs well , worked actors recognize movie real-life drama ! possible spoiler alert : one great scene , two leads , actors , _discussing_ _discuss_ something personal , something entirely 'out-of-script ' , another actor , start making lines , rehearsing , critiquing 's performance.since movie appeared , , '91 , become fashionable , especially tv . hardly anyone done well .
Actual Sentiment: positive
Predicted Sentiment: 0.17161172161172158
----------------------------------------------------------------------------------------------------
REVIEW: shower story loyalty , unstoppable advance modern world detr

In [ ]:
pre_process_corpus(test_reviews[1500])

In [48]:
test_reviews[1500]

"sure comment contains actual spoiler playing safe , read seen movie.i adore movie , everyone work -- point . spent large part working life cinema , without actor . people _sung_ heroes movie : gaffers , pullers , on-air directors , lighters writers , costume people etc etc , whole thing told point view , least great extent . actors nuts self-absorbed point absurdity , movie spoofs well , worked actors recognize movie real-life drama ! possible spoiler alert : one great scene , two leads , actors , _discussing_ _discuss_ something personal , something entirely 'out-of-script ' , another actor , start making lines , rehearsing , critiquing 's performance.since movie appeared , , '91 , become fashionable , especially tv . hardly anyone done well ."

In [ ]:
#AFinn

In [66]:
from afinn import Afinn
afn = Afinn(emoticons = True)

In [69]:
for review, sentiment in zip(test_reviews[sample_review_ids],test_sentiments[sample_review_ids]):
  print('REVIEW:',review)
  print('Actual Sentiment:',sentiment)
  print("Predicted Sentiment:", afn.score(review))
  print('-'*100)

REVIEW: I'm not sure that this comment contains an actual spoiler, but I'm playing it safe, so don't read this if you haven't seen the movie.<br /><br />I adore this movie, and so does everyone I work with -- and that is the point. I spent a large part of my working life in cinema, without being an actor. Such people are the _sung_ heroes of this movie: the gaffers, the pullers, the on-air directors, the lighters and writers, the costume people etc etc, and the whole thing is told from their point of view, at least to a great extent. Most actors are nuts and self-absorbed to the point of absurdity, which is what this movie spoofs so well, but you have to have worked with actors to recognize that this movie is real-life drama! Possible spoiler alert: in one great scene, the two leads, both actors, are _discussing_ how to _discuss_ something personal, something entirely 'out-of-script', with another actor, and they start making up lines, rehearsing them, and critiquing each other's perfo

In [ ]:
#Vader - -4 to 4
#Adv - It is classified by 10different users 
#Compound score - -1 to 1 

In [70]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

In [76]:
def sentiment_analysis_vader(review, threshold = 0.2,verbose = False):
  analyser = SentimentIntensityAnalyzer()
  scores = analyser.polarity_scores(review)
  agg_score = scores['compound']
  final_sentiment = 'positive' if agg_score > threshold else 'negative'
  return scores,final_sentiment

In [77]:
scores, sentiments = sentiment_analysis_vader(test_reviews[1500], threshold = 0.2,verbose = False)

In [79]:
sentiments

'positive'

In [80]:
scores

{'compound': 0.9884, 'neg': 0.021, 'neu': 0.816, 'pos': 0.162}